In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

# Adjust display options
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full column width

In [ ]:
import pandas as pd

# Load your data
df = pd.read_csv('your_dataset.csv')

# Define a simple function for annotation
def annotate_effect(effect_text):
    if 'increase' in effect_text or 'boost' in effect_text:
        return 'positive' 'decrease' in effect_text or 'reduce' in effect_text
    
        return 'negative'
    
        return 'neutral' # Apply the annotation function to the 'Effect' column
    
df['Effect_Label'] = df['Effect'].apply(annotate_effect)

# Save the annotated data
df.to_csv('annotated_dataset.csv', index=False)

print(df[['Effect', 'Effect_Label']].head())

In [19]:
df = pd.read_excel('../Data/legendary_lord_defeat_traits.xlsx')

df['Effect'][25:50]

25                                                                                 Attribute: Expert Charge Defence  Melee defence: +3 (Lord's army)
26                                                                                            Magic item drop chance: +10%  Armour: +5 (Lord's army)
27                                                                                            Winds of Magic cost: -20% for all spells (Lord's army)
28                                                                                                           Armour-piercing damage: +20 Armour: +10
29                                                             Ambush Success Chance: +20% ( Lord's army) Ambush Defence Chance: +20% ( Lord's army)
30                                                                                                                          Physical resistance: 10%
31                                              Diplomatic relations: +10 with High Elves  Attribute: Caus